# *OpenScope: P3*
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThsuIB8o1pQXxrorbe2CMAr7O_wfn5B_wQBg&s" width="380" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [1]:
import contextlib
import time
import shutil
import os

import np_config
import np_jobs
import np_logging
import np_services
import np_session
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_P3 as P3

from np_services.resources.zro import ZroError 
from zmq.error import ZMQBaseError

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [2]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [3]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [24]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select workflow
Re-run cell this cell if mouse ID is changed

In [25]:
np_workflows.mtrain_widget(mouse)

Select type of workflow to run

In [26]:
selected_workflow = P3.P3_workflow_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [27]:
#P3.validate_selected_workflow(selected_workflow.session, mouse)

experiment: P3.Ephys | P3.Hab = P3.new_experiment(mouse, user, selected_workflow.session)
session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, P3.Hab)

11:48 | web | INFO | Ephys(1484427425_366122_20260106) created


***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [8]:
np_workflows.check_hardware_widget()

In [9]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [10]:
with contextlib.suppress(ZroError, ZMQBaseError):
    experiment.initialize_and_test_services()

09:15 | np_services.utils | INFO | mouse_director is running on DT900490
09:15 | np_services.proxies | INFO | Proxy(MouseDirector) initialized: ready for use
09:15 | np_services.utils | INFO | sync_device is running on W10DT26AD0025
09:15 | np_services.proxies | INFO | Proxy(Sync) initialized: ready for use
09:15 | np_services.proxies | INFO | VideoMVR initialized: ready for use
09:16 | np_services.proxies | INFO | ImageMVR initialized: ready for use
09:16 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
09:16 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
09:16 | np_services.utils | INFO | camstim_agent is running on W10DT713942
09:16 | np_services.utils | INFO | phidget_server is running on W10DT713942
09:16 | np_services.proxies | INFO | Proxy(SessionCamstim) initialized: ready for use
09:16 | np_services.open_ephys | INFO | OpenEphys | Initializing
09:16 | np_services.open_ephys | INFO | OpenEphys | Testing
09:16 | np_services.open_ep

***
## Dip probes

In [11]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)


    - scan barcode or enter ID number for the dye used
    - change dye description if incorrect (DiI, DiO)
    - increment number of times probes were dipped this session
    - hit `Save` to store info in platform.json
    


## Photodoc of probes in dye

In [12]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image1-left.png')
    else:
        np_workflows.photodoc_widget('pre_experiment_surface_image')

1476893236_820462_20251120_surface-image1-left.png


## Probe depths in dye

In [13]:
if not hab:
    np_workflows.probe_depth_widget(session)

09:29 | np_workflows.shared.widgets | WARNING | No photodocs have been captured yet.


***
***
# **With mouse on stage**
## Before lowering cartridge

In [11]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

### Set zoom to 4.0 for photodocs of brain
- focus on the brain surface

## Photodoc of brain (tap probes if hab Day1 or Day2)

In [15]:
platform_json.CartridgeLowerTime = npxc.now()
if platform_json.rig_id == 'NP.1':
    print(str(session) + '_surface-image2-left.png')
else:    
    np_workflows.photodoc_widget('brain_surface_image')

1476893236_820462_20251120_surface-image2-left.png


***
## ISI map

In [16]:
np_workflows.isi_widget(mouse.lims)

ISI map found for 820462:
\\allen\programs\mindscope\production\openscope\prod0\specimen_1466328801\isi_experiment_1469043838\1469043838_target_map.tif


***
## Probe insertion

In [12]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()

***
## Photodoc before advancing probes

In [18]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image3-left.png')
    else:
        np_workflows.photodoc_widget('pre_insertion_surface_image')

1476893236_820462_20251120_surface-image3-left.png


***
## Settle timer & insertion notes

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

In [13]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [20]:
if not hab:
    np_workflows.print_countdown_timer(minutes=.1 if experiment.workflow.value == 'pretest' else 10)

***
## Photodoc after probes settled, before experiment

In [21]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image4-left.png')
    else:
        np_workflows.photodoc_widget('post_insertion_surface_image')

1476893236_820462_20251120_surface-image4-left.png


In [14]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [15]:
last_exception = Exception()
attempts = 3
while attempts:
    np_logging.getLogger().info('Waiting for recorders to finish processing') 
    while not all(r.is_ready_to_start() for r in experiment.recorders):
        time.sleep(1)
    np_logging.getLogger().info('Recorders ready')     
    try:
        experiment.start_recording()
    except AssertionError as exc:
        np_logging.getLogger().info('`experiment.start_recording` failed: trying again')
        attempts -= 1
        last_exception = exc              # exc only exists within the try block
    
    else:
        break
else:
    np_logging.getLogger().error(f'`experiment.start_recording` failed after multiple attempts', exc_info=last_exception)
    raise last_exception

09:58 | root | INFO | Waiting for recorders to finish processing
09:58 | root | INFO | Recorders ready
09:58 | np_services.proxies | INFO | Sync | Starting recording
09:59 | np_services.proxies | INFO | Sync | Verified: file on disk is increasing in size
09:59 | np_services.proxies | INFO | VideoMVR | Starting recording
09:59 | np_services.proxies | INFO | VideoMVR | Verified: 3 cameras recording to disk
09:59 | np_services.open_ephys | INFO | OpenEphys | Starting recording
09:59 | np_services.open_ephys | INFO | OpenEphys | Verified files are increasing in size for all Record Nodes


***
## Start stimulus
mapping | active | injection | active | mapping | opto


Script and params determined by MTrain stage

In [16]:
with contextlib.suppress(ZroError, ZMQBaseError):
    experiment.run_stim()
    

09:59 | web | INFO | Started session EPHYS_SESSION1_PROD_P3_SENSORYMOTOR
11:27 | np_services.proxies | INFO | Finalizing SessionCamstim


***
## Stop recording

In [18]:
with contextlib.suppress(ZroError, ZMQBaseError):
    experiment.stop_recording_after_stim_finished()

11:27 | np_services.open_ephys | INFO | OpenEphys | Stopping recording
11:27 | np_services.proxies | INFO | VideoMVR | Stopped recording
11:28 | np_workflows.shared.base_experiments | WARNING | Waiting additional 4 s for MVR to finish writing...
11:28 | np_services.proxies | INFO | Sync | Stopped recording


***
## Before removing probes

In [19]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image5-left.png')
    else:
        np_workflows.photodoc_widget('post_stimulus_surface_image')

1481489869_366122_20251217_surface-image5-left.png


***
## After fully retracting probes

In [47]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image6-left.png')
    else:
        np_workflows.photodoc_widget('post_experiment_surface_image')

1484427425_366122_20260106_surface-image6-left.png


***
## After raising cartridge

In [20]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [21]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

11:40 | np_services.proxies | ERROR | Could not get camera status from DT900490
11:40 | np_services.open_ephys | INFO | OpenEphys | Finalizing
11:40 | np_services.open_ephys | INFO | OpenEphys | Starting recording
11:40 | np_services.open_ephys | INFO | OpenEphys | Stopping recording
11:40 | np_services.proxies | INFO | Finalizing SessionCamstim
11:40 | np_services.proxies | INFO | SessionCamstim added new data: []
11:40 | npc_sync.sync | INFO | **********************************************************************
11:40 | npc_sync.sync | INFO | Quick stats for line: vsync_stim
11:40 | npc_sync.sync | INFO | Label: vsync_stim
11:40 | npc_sync.sync | INFO | Bit: 2
11:40 | npc_sync.sync | INFO | Data points: 4355389
11:40 | npc_sync.sync | INFO | Total transitions: 518642
11:40 | npc_sync.sync | INFO | Rising edges: 259321
11:40 | npc_sync.sync | INFO | Falling edges: 259321
11:40 | npc_sync.sync | INFO | Average period: 0.016680587654587593
11:40 | npc_sync.sync | INFO | Minimum period:

Output()

11:41 | np_services.proxies | INFO | Validated 6 video/info files without sync
11:41 | np_services.open_ephys | INFO | OpenEphys | Validating
11:41 | npc_ephys.openephys | INFO | No compressed data found for \\W10DT713843\A\1481489869_366122_20251217\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeB-AP
11:41 | npc_ephys.openephys | INFO | No compressed data found for \\W10DT713843\A\1481489869_366122_20251217\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeB-LFP
11:41 | npc_ephys.openephys | INFO | No compressed data found for \\W10DT713843\A\1481489869_366122_20251217\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeC-AP
11:41 | npc_ephys.openephys | INFO | No compressed data found for \\W10DT713843\A\1481489869_366122_20251217\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeC-LFP
11:41 | npc_ephys.openephys | INFO | No compressed data found for \\W10DT713843\A\1481489869_366122_20251217\Rec

## Copy data

In [41]:
# copy motor-locs from vidmon -> np-exp
source_path = r"\\DT900490\C$\MPM_data\log.csv" 

renamed = f'{experiment.session.folder}.motor-locs.csv'

try:
    shutil.copy2(source_path, experiment.session.npexp_path / (renamed)
    #shutil.copy2(source_path, destination_path)
    print(f"File successfully copied from {source_path} to {experiment.session.npexp_path / (renamed }")
except FileNotFoundError:
    print("Error: The source file or destination directory was not found")
except Exception as e:
    print(f"An error occurred: {e}")

File successfully copied from \\DT900490\C$\MPM_data\log.csv to \\allen\programs\mindscope\workgroups\np-exp\1484427425_366122_20260106\1484427425_366122_20260106.motor-locs.csv


In [59]:
# copy images from  vidmon --> np-exp
source_path = r"\\DT900490\C$\Users\svc_neuropix\Desktop\ecephys_photodocs"
os.chdir(source_path)
for pic in os.listdir():
    if pic.__contains__(experiment.session.folder):
        print('Found image ', pic)
        pic_path = os.path.join(source_path, pic)
        try:
            shutil.copy2(pic_path, experiment.session.npexp_path / (pic))
            print(f"File successfully copied from {pic_path} to {experiment.session.npexp_path / (pic)}")
        except FileNotFoundError:
            print("Error: The source file or destination directory was not found. Ensure the network share is accessible and the path is correct.")
        except Exception as e:
            print(f"An error occurred: {e}")
        

Found image  1484427425_366122_20260106_surface-image2-left.png
File successfully copied from \\DT900490\C$\Users\svc_neuropix\Desktop\ecephys_photodocs\1484427425_366122_20260106_surface-image2-left.png to \\allen\programs\mindscope\workgroups\np-exp\1484427425_366122_20260106\1484427425_366122_20260106_surface-image2-left.png
Found image  1484427425_366122_20260106_surface-image6-left.png
File successfully copied from \\DT900490\C$\Users\svc_neuropix\Desktop\ecephys_photodocs\1484427425_366122_20260106_surface-image6-left.png to \\allen\programs\mindscope\workgroups\np-exp\1484427425_366122_20260106\1484427425_366122_20260106_surface-image6-left.png


In [22]:
experiment.copy_files()

11:41 | np_workflows.shared.base_experiments | INFO | Sync | Copying files {WindowsPath('//W10DT26AD0025/c$/ProgramData/AIBS_MPE/sync/data/20251217T095845.h5')}
11:41 | np_workflows.shared.base_experiments | INFO | VideoMVR | Copying files {WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Behavior_20251217T095900.json'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Eye_20251217T095901.mp4'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Behavior_20251217T095900.mp4'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Face_20251217T095901.json'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Eye_20251217T095901.json'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Face_20251217T095901.mp4')}
11:41 | np_workflows.shared.base_experiments | INFO | SessionCamstim | Copying files {WindowsPath('//W10DT713942/c$/ProgramData/AIBS_MPE/camstim/data/251217095924_366122_7186f610-ffc6-4e3c-9943-bcfefd5e4cf0.pkl'), WindowsPath('//W10DT713942

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

In [23]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)

In [24]:
np_jobs.PipelineQCQueue().add_or_update(session, priority=99)